## Example Analysis

In [2]:
import os
from PyFinModeler import (
    Company, AssumptionSet, ForecastRule, ForecastModel,
    DividendDiscountModel, ValuationSummaryReport, ScenarioModel,
    ChartGenerator, BorsdataCollector, BorsdataKPICollector,
    FinancialItem, FinancialItemType, create_markdown_table
)
from IPython.display import Markdown, display

### Step 1 Load API Key

In [3]:
api_key = os.environ.get("BORSDATA_API_KEY")
if not api_key:
    raise ValueError("API Key not found. Please set BORSDATA_API_KEY.")

### Step 2: Initialize collectors

In [4]:
collector = BorsdataCollector(api_key=api_key)
kpi_collector = BorsdataKPICollector(api_key=api_key)

### Step 3: Fetch Company (Atlas Copco B)

In [5]:
company, company_id, company_info = collector.fetch_company_by_name("Atlas Copco B", report_type="year")

if company is None:
    print("Company not found.")
    exit()
else:
    display(Markdown(create_markdown_table(company_info)))

| Key | Value |
|:----|:------|
| insId | 1605 |
| name | Atlas Copco B |
| urlName | atlas-copco-b |
| instrument | 3 |
| isin | SE0017486897 |
| ticker | ATCO B |
| yahoo | ATCO-B.ST |
| sectorId | 5 |
| marketId | 1 |
| branchId | 23 |
| countryId | 1 |
| listingDate | 1973-01-01T00:00:00 |
| stockPriceCurrency | SEK |
| reportCurrency | SEK |


### Step 4: Fetch real KPIs from Nordic endpoint

In [6]:
# Step 4: Fetch real KPIs from Nordic endpoint

# Fetch Net Income Margin (Net Margin %)
try:
    net_income_margin_values = kpi_collector.fetch_kpis(company_id, kpi_id=30, report_type="year", price_type="mean")
    net_income_margin = list(net_income_margin_values.values())[0]
except Exception as e:
    print(f"Warning: Could not fetch Net Income Margin. Error: {str(e)}")
    net_income_margin = 0.20  # fallback

# Fetch Payout Ratio %
try:
    payout_ratio_values = kpi_collector.fetch_kpis(company_id, kpi_id=20, report_type="year", price_type="mean")
    payout_ratio = list(payout_ratio_values.values())[0] / 100
except Exception as e:
    print(f"Warning: Could not fetch Payout Ratio. Error: {str(e)}")
    payout_ratio = 0.5

# Fetch Number of Shares
try:
    shares_values = kpi_collector.fetch_kpis(company_id, kpi_id=61, report_type="year", price_type="mean")
    shares_outstanding = list(shares_values.values())[0]
except Exception as e:
    print(f"Warning: Could not fetch Number of Shares. Error: {str(e)}")
    shares_outstanding = 100_000_000

In [8]:
net_income_margin_values, payout_ratio_values, shares_values

({2024: 16.847999572753906,
  2023: 16.239999771118164,
  2022: 16.61199951171875,
  2021: 16.34600067138672,
  2020: 14.810999870300293,
  2019: 15.923999786376953,
  2018: 17.1299991607666,
  2017: 14.319999694824219,
  2016: 11.770999908447266,
  2015: 11.468999862670898},
 {2024: 49.18000030517578,
  2023: 48.611000061035156,
  2022: 47.71099853515625,
  2021: 51.04499816894531,
  2020: 60.03499984741211,
  2019: 51.46500015258789,
  2018: 46.79800033569336,
  2017: 50.9739990234375,
  2016: 69.30599975585938,
  2015: 65.45600128173828},
 {2024: 4874.2001953125,
  2023: 4873.0,
  2022: 4870.0,
  2021: 4870.7998046875,
  2020: 4861.60009765625,
  2019: 4858.7998046875,
  2018: 4854.0,
  2017: 4856.0,
  2016: 4864.0,
  2015: 4869.60009765625})